# Models Training 3
In this notebook we are exploring Word Embeddings

## Imports

In [ ]:
!pip install -q spacy
!python -m spacy download es_core_news_md

In [25]:
import numpy as np
import pandas as pd
import spacy
import gensim
from sklearn.ensemble import RandomForestClassifier


# import custom helper module
import importlib
import helpers
importlib.reload(helpers)


2024-10-02 22:40:15.388711: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-02 22:40:15.389136: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-02 22:40:15.389293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

<module 'helpers' from '/notebooks/helpers.py'>

## Load dataset

In [28]:
X_train, X_test, y_train, y_test = helpers.load_dataset("TRAINING_DATA.txt")

# print message
helpers.print_text(X_train, y_train)


No pickle file found. Loading and cleaning dataset.
----------------------------------------------------------------------------------------------------
[9522] tan triste parejas hace años decidan separarse --> 1
----------------------------------------------------------------------------------------------------


## Tokenize data

In [8]:
# X_train_tok = [nltk.word_tokenize(text) for text in X_train]


In [ ]:
# from gensim.models import Word2Vec
# import gensim.downloader as api
# import sys
# sys.path.append('I:\\ai_models\\spanish_word2vec')

# import os

# model_path = r"I:\ai_models\spanish_word2vec\complete.model"
# print(os.path.exists(model_path))  # Should return True if the file exists

# # train Word2Vec on tokenized dataset
# # w2v_model = Word2Vec(sentences=X_train_tok, vector_size=100, window=5, min_count=1, workers=4)

# # save model
# # w2v_model.save("word2vec.model")
# model_path = r'file:///I:/ai_models/spanish_word2vec/complete.model'
# w2v_model = Word2Vec.load(model_path)


## Loading spaCy and Word Embeddings

In [29]:
# Load the Spanish language model
nlp = spacy.load("es_core_news_md")

# Example text
text = "Esta es una oración de ejemplo."

# Process the text with spaCy to obtain word embeddings
doc = nlp(text)

# Access word embeddings for each token
for token in doc:
    print(token.text, token.vector)  # `token.vector` gives the word embedding



Esta [ 5.6426e+00 -2.2479e-01  2.4950e+00  1.2205e+00  5.6342e+00 -4.4999e+00
 -4.5745e-01  8.7406e-01  3.1076e+00 -1.3380e+00  1.0341e+00  7.2001e+00
  4.6250e-01 -4.8635e+00  3.7515e+00  6.7590e-03 -8.3981e-02 -4.4031e+00
  6.2830e+00  1.3133e+00  1.6437e+00 -2.3727e-02 -3.3668e-01  2.7206e+00
  2.4771e+00  4.4419e+00  1.1142e+00  2.7833e+00 -4.4402e+00  6.9588e-01
  9.1691e-01  3.9655e+00 -2.9583e-01 -4.8001e+00 -3.2515e+00  1.6547e+00
  2.6210e+00  5.4598e-01  3.2995e+00  1.4479e+00  1.3022e+00  9.5587e-01
 -1.8186e+00 -3.0594e+00  1.0719e+00  4.2435e+00  3.4267e+00  7.8943e+00
 -2.8035e+00 -1.0325e+00  1.0798e+00  4.3054e+00  3.8553e+00 -4.1107e-01
 -3.3289e+00  3.0916e+00 -1.2414e+00 -4.1578e+00 -5.4438e+00  7.7858e-01
  4.7921e+00  7.7843e-01  4.6567e-01 -3.6170e-01  4.1328e+00  3.7598e-01
  1.2302e-01 -1.4008e+00 -2.8097e+00  1.2602e+00 -4.9633e+00  4.6132e-03
  2.6106e+00 -1.6016e+00 -4.5058e+00 -9.8519e-01 -3.0157e+00 -5.8176e+00
  2.0709e-01  2.0357e-01  3.7225e+00 -1.4230e+

In [32]:
# Function to get the average word vector of a sentence
def get_sentence_embedding(sentence):
    doc = nlp(sentence)
    # Collect embeddings of all tokens that have vectors
    vectors = [token.vector for token in doc if token.has_vector]
    
    if len(vectors) > 0:
        # Average the vectors if valid embeddings exist
        return np.mean(vectors, axis=0)
    else:
        # If no valid tokens, return a zero vector of size 300 (or whatever your embedding size is)
        return np.zeros(nlp.vocab.vectors_length)

# Example of sentence embedding
sentence_embedding = get_sentence_embedding("Esta es una oración de ejemplo.")
print(sentence_embedding.shape)
print(sentence_embedding[:10])

(300,)
[ 1.2771758  -0.24261244  0.5443317  -0.5648171   1.6554718  -2.8524573
 -0.09760858  1.0279129   1.2620614  -1.4583185 ]


In [33]:
# encoding our dataset
X_train_embeddings = np.array([get_sentence_embedding(sentence) for sentence in X_train])
X_test_embeddings = np.array([get_sentence_embedding(sentence) for sentence in X_test])

# Initialize and train classifier
clf = RandomForestClassifier()
clf.fit(X_train_embeddings, y_train)

# Evaluate on test set
accuracy = clf.score(X_test_embeddings, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.3326633165829146


In [34]:
from sklearn.neural_network import MLPClassifier

# Initialize an MLP
mlp = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)

# Train the MLP
mlp.fit(X_train_embeddings, y_train)

# Evaluate
accuracy = mlp.score(X_test_embeddings, y_test)
print(f"MLP Accuracy: {accuracy}")

MLP Accuracy: 0.35678391959798994


In [35]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Train the SVM model
svm_model = SVC(kernel='linear', C=1.0, random_state=42)  # You can experiment with the 'kernel' parameter
svm_model.fit(X_train_embeddings, y_train)

# Make predictions on the test set
y_pred = svm_model.predict(X_test_embeddings)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {accuracy}")

# Print a detailed classification report
print(classification_report(y_test, y_pred))


SVM Accuracy: 0.5262981574539364
              precision    recall  f1-score   support

           0       0.53      0.53      0.53      1493
           1       0.53      0.52      0.52      1492

    accuracy                           0.53      2985
   macro avg       0.53      0.53      0.53      2985
weighted avg       0.53      0.53      0.53      2985

